In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import curve_fit
import time
from scipy.stats import gaussian_kde
from tqdm import tqdm
from scipy.optimize import fsolve
import math

In [2]:


# Initialize an empty DataFrame to hold all the testing degradation data
all_test_data = pd.DataFrame()

# Path to the directory containing the item files
items_directory = './robot_maintenance/testing_data/sent_to_student/group_0'
#items_directory = './robot_maintenance/training_data/degradation_data'

# Loop through all item files and concatenate data for relevant items
for item_id in range(50):
    file_path = os.path.join(items_directory, f'testing_item_{item_id}.csv')
    if os.path.exists(file_path):  # Check if the file exists
        item_data = pd.read_csv(file_path)
        # Add an 'item_id' column to the DataFrame
        item_data['item_id'] = item_id
        # Concatenate this item's data into the aggregate DataFrame
        all_test_data = pd.concat([all_test_data, item_data], ignore_index=True)

# Now all_test_data contains the aggregated testing data for all items
all_test_data


,time (months),crack length (arbitary unit),item_id
0,0,0.003686,0
1,1,-0.014691,0
2,2,-0.048912,0
3,3,-0.000898,0
4,4,-0.007360,0
...,...,...,...
662,6,0.048160,49
663,7,0.119816,49
664,8,0.075937,49
665,9,0.017686,49


In [3]:
class pf_class:
    '''
    This is the definition of particle filetering class.

    Methods:
    - __init__: Initialization function.
    - state_estimation(): Implement the particle filtering.
    - rul_prediction(): Predict RUL.
    - resample(): Perform resampling in state estimation.
    - get_state_estimation(): This function allows estimating the mean and CI based on samples and weights.
    '''

    def __init__(self, Ns, t, nx, gen_x0, sys, obs, p_yk_given_xk, gen_sys_noise, degradation_path=[], initial_outlier_quota=3):
        '''
        Initialization function of the PF class.

        Args:
        - Ns: Particle size.
        - t: An array of the measurement time.
        - nx: Number of state variables, a scalar value.
        - w: weights   (Ns x T), where T is the total number of measurements.
        - particles: particles (nx x Ns x T).
        - gen_x0: function handle of a procedure that samples from the initial pdf p_x0 to create initial particles.
        - p_yk_given_xk: function handle of the observation likelihood PDF p(y[k] | x[k]).
        - gen_sys_noise: function handle of a procedure that generates system noise.
        - sys: function handle to process equation.
        - obs: function handle to observation equation.
        - degradation_path: function handle to calcualte the degradation measures.
        - initial_outlier_quota: The number of concecutive outliers allowed, before reinitiating the particles.
        '''
        self.k = 0 # Current step.
        self.Ns = Ns # Particle size.
        T = len(t) # Get the number of measuring points.
        # Memory assignment.
        self.w = np.zeros((Ns, T)) # Weights.
        self.t = t # Observation time
        self.particles = np.zeros((nx, Ns, T)) # Particles.
        # Store the function handles of the state space model.
        self.gen_x0 = gen_x0
        self.sys = sys
        self.obs = obs
        self.p_yk_given_xk = p_yk_given_xk
        self.gen_sys_noise = gen_sys_noise
        self.outlier_quota = initial_outlier_quota
        self.initial_outlier_quota = initial_outlier_quota
        self.degradation_path = degradation_path


    def state_estimation(self, yk, resampling_strategy='multinomial_resampling'):
        """
        This function implement a generic particle filter to estimate the system states. Note: when resampling is performed on each step this algorithm is called
        the Bootstrap particle filter.

        Usage:
        xhk = pf.state_estimation(yk, resamping_strategy)
        This function has to be called sequentially, at k=1,2,\cdots,

        Args:
        - yk   = observation vector at time k (column vector)
        - resampling_strategy = resampling strategy. Set it either to
                                'multinomial_resampling' (default) or 'systematic_resampling'

        Outputs:
        * xhk   = estimated state

        Reference: Arulampalam et. al. (2002).  A tutorial on particle filters for online nonlinear/non-gaussian bayesian tracking. IEEE Transactions on Signal Processing. 50 (2). p 174--188

        This function was developed based on an original verision in Matlab programmed by: Diego Andres Alvarez Marin (diegotorquemada@gmail.com), February 29, 2012.
        """
        # Get the current step.
        t = self.t
        k = self.k
        if k == 0:
            raise ValueError('error: Cannot have only one step!')

        # Initialize variables
        Ns = self.Ns  # number of particles
        # If it is the first data point, we need to create the initial particles.
        if k == 1:
            self.particles[:, :, 0] = self.gen_x0(Ns)  # Generate the initial particles.
            self.w[:, 0] = np.repeat(1 / Ns, Ns)  # All particles have the same weight initially.

        # Separate memory
        xkm1 = self.particles[:, :, k-1]  # extract particles from last iteration;
        wkm1 = self.w[:, k-1]  # weights of last iteration
        xk = np.zeros_like(xkm1)  # Initial values for the current particles.
        wk = np.zeros_like(wkm1)  # _ for the current weights.

        # Algorithm 3 of Ref [1].
        uk = self.gen_sys_noise(Ns) # Generate the process noise.
        for i in range(Ns): # For each particle, predict its state in the next time instant.
            xk[:, i] = self.sys(t[k], xkm1[:, i], uk[:, i]) # This is the state equation.
            wk[i] = wkm1[i] * self.p_yk_given_xk(yk, xk[:, i]) # Update the weights (when using the PRIOR pdf): eq 63, Ref 1.
        # Handle exception:
        if sum(wk) == 0: # If sum(wk)==0: Keep the previous weigths.
            if self.outlier_quota == 1:
                # print(f'Reinitiate the particles: k={k}')
                xk = self.gen_x0(Ns, t[k])
                wk = np.repeat(1 / Ns, Ns)
                self.outlier_quota = self.initial_outlier_quota
            else:
                # print(f'Smoothing due to weight NaN: k={k}')
                for i in range(Ns):
                    xk[:, i] = self.sys(t[k], xkm1[:, i], np.zeros(xkm1.shape[0]-1))
                wk = wkm1
                self.outlier_quota -= 1
        else: # Here we scrape the outlier.
            y_tmp = xkm1[-1, :] # This is the degradation estimation of each particles.
            y_w = wkm1
            y_mean = self.sys(t[k], np.matmul(xkm1, wkm1), np.zeros(xkm1.shape[0]-1))[-1]
            _, y_bands = self.get_state_estimation(y_tmp, y_w) # Get the 90% CI of the estimation.
            interval_width = y_bands[1]-y_bands[0]
            # A outlier is defined as exceeding 1.5 interval_width from the upper and lower bound.
            margin = 1
            if (yk > y_mean+margin*interval_width) | (yk < y_mean-margin*interval_width):
                if self.outlier_quota == 1:
                    # print(f'Reinitiate the particles: k={k}')
                    xk = self.gen_x0(Ns, t[k])
                    wk = np.repeat(1 / Ns, Ns)
                    self.outlier_quota = self.initial_outlier_quota
                else:
                    # print(f'Smoothing due to outlier: k={k}')
                    for i in range(Ns):
                        xk[:, i] = self.sys(t[k], xkm1[:, i], np.zeros(xkm1.shape[0]-1))
                    wk = wkm1
                    self.outlier_quota -= 1
            else:
                wk = wk/sum(wk)
                self.outlier_quota = self.initial_outlier_quota

        # Resampling if necessary.
        resample_percentaje = 0.50
        Nt = resample_percentaje * Ns
        # Calculate effective sample size: eq 48, Ref 1
        Neff = 1 / sum(wk**2)
        if Neff < Nt:
            # print('Resampling ...')
            xk, wk = self.resample(xk, wk, resampling_strategy)

        # Compute estimated state
        xhk = np.matmul(xk, wk)

        # Store new weights and particles
        self.w[:, k] = wk
        self.particles[:, :, k] = xk

        return xhk


    def resample(self, xk, wk, resampling_strategy='multinomial_resampling'):
        '''
        This function implements the resampling of PF.

        Args:
        - xk: The original particles.
        - wk: Weights.
        - resampling_strategy: A string of the resamping strategy:
            -- 'multinomial_resampling' (default): Sampling with replacement.
            -- 'systematic_resampling': Latin hypercube sampling

        Outputs:
        - xk: The particles after resampling.
        - wk: The new weights.
        '''
        Ns = len(wk)  # Ns = number of particles

        if resampling_strategy == 'multinomial_resampling': # Sampling with replacement.
            with_replacement = True
            idx = np.random.choice(np.arange(Ns), Ns, p=wk, replace=with_replacement)
        elif resampling_strategy == 'systematic_resampling':
            # this is performing latin hypercube sampling on wk
            edges = np.minimum(np.cumsum(wk), [1]*len(wk))  # protect against accumulated round-off
            edges[-1] = 1  # get the upper edge exact
            u1 = np.random.random()/Ns
            # this works like the inverse of the empirical distribution and returns
            # the interval where the sample is to be found
            _, idx = np.histogram(np.arange(u1, 1, 1/Ns), edges)
        else:
            raise ValueError('Resampling strategy not implemented!')

        xk = xk[:, idx]  # extract new particles
        wk = np.ones(Ns)/Ns  # now all particles have the same weight

        return xk, wk


    def rul_prediction(self, threshold, idx_pred, t, max_RUL=145, alpha = .1):
        '''
        This function predicts the RUL based on the result of the PF.

        Args:
        - threshold: The failure threshold. Failure is defined when the degradation measures < threshold.
        - t_pred: An array representing the time horizon in which you would like to make prediction. Note:
            -- t_pred should cover t, i.e., the first part of the t_pred should contain all the measurement time instants.
            -- The second part of t_pred contains the times corresponds to the prediction steps in particle filtering. For example, suppose a step in PF represents 10 time units and the observation ends at t=100.
            If you wish to predict the degradation trajectory in the next 10 time step, then, t_pred = [t, 110, 120, \cdots, 210].
        - idx_pred: An array containing the indexes in t_pred, that represents the time instants you wish to perform RUL prediction.
            For example, t_pred = [10, 20, 30, \cdots, 100], and you have measurement data for the first three points $10, 20$ and $30$.
            If you wish to predict RUL at these three time instants, then you should set idx_pred = [0, 1, 2]. By doing so, three seperate RUL predictions will be performed. Each prediction will only use the degradation
            measurements before it.
        - max_RUL: When the search for RUL ends without finding the failure time, we set the RUL = max_RUL. Default value is $145$.
        - alpha: Confidence level for calculating the confidence interval. Default value is .1.

        Outputs:
        - rul_mean: An array containing the mean value of the predicted RUL. Shape (n_pred), where n_pred is the number of predictions.
        - rul_bands: An ndarry containting the upper and lower CI for the mean RUL. Shape: (n_pred, 2)
        - rul: An ndarray containing the predicted RULs for all the particles. Shape (Ns, n_pred) where Ns is the number of particles.
        - rul_weights: An ndarry containing the weights for the predicted RULs. Shape (Ns, n_pred).
        - deg_pred_mean: A list of n_pred element. Each element is the predicted degradation path at a given prediction point.
        - deg_pred_bands: The CI of the previous variable.
        '''
        # Initialize the variables.
        n_pred = len(idx_pred)
        rul = max_RUL*np.ones((self.Ns, n_pred))
        rul_mean = np.zeros(n_pred)
        rul_bands = np.zeros((n_pred, 2))
        rul_weights = np.zeros((self.Ns, n_pred))

        deg_pred_mean = []
        deg_pred_bands = []

        # Do a loop to make RUL predictions:
        for i in tqdm(range(n_pred)):
            idx_pred_i = idx_pred[i] # Index of the prediction instant.
            rul_weights[:, i] = self.w[:, idx_pred_i] # Get the weights.

            # For each particle, use the degradation model to calculate the RUL.
            for j in range(self.Ns):
                x_cur = self.particles[:, j, idx_pred_i] # Get the current particles.
                # Define a equation: Degration(t)=0, and solve for t.
                hdl_eq = lambda tt: self.degradation_path(x_cur, tt) - threshold
                ttf_run = fsolve(hdl_eq, t[idx_pred_i])
                # Get RUL.
                rul[j, i] = math.floor(ttf_run) + 1 - t[idx_pred_i]
                if rul[j, i] > max_RUL:
                    rul[j, i] = max_RUL
                elif rul[j, i] < 0:
                    rul[j, i] = 0
            # Calculate the mean and CI for the predicted RUL.
            rul_mean[i], rul_bands[i, :] = self.get_state_estimation(rul[:, i], rul_weights[:, i], alpha=alpha)

            # Degradation state prediction:
            n_t = math.floor(rul_mean[i]) + 10
            deg_pred = np.zeros((self.Ns, n_t))
            for j in range(self.Ns):
                x_cur = self.particles[:, j, idx_pred_i] # Get the current particles.
                tt = np.arange(t[idx_pred_i]+1, t[idx_pred_i]+1+n_t)
                deg_pred[j, :] = self.degradation_path(x_cur, tt)
            # Get the mean and bands of the degradation prediction.
            deg_mean = np.zeros(n_t)
            deg_bands = np.zeros((n_t, 2))
            for j in range(n_t):
                deg_mean[j], deg_bands[j, :] = self.get_state_estimation(deg_pred[:, j], rul_weights[:, i], alpha=alpha)
            deg_pred_mean.append(deg_mean)
            deg_pred_bands.append(deg_bands)

        return rul_mean, rul_bands, rul, rul_weights, deg_pred_mean, deg_pred_bands


    def get_state_estimation(self, x_sample, weights, alpha=.1):
        '''
        This function estimates the mean and CI based on particles with weights.

        Args:
            - x: An array of the particles.
            - weights: weights.
            - alpha: confidence level, default is .1.

        Outputs:
            - est_mean: The mean value of the estimate.
            - est_bands: Confidence bands.
        '''
        # Get the mean estimation
        est_mean = np.dot(x_sample, weights)

        # Get the CI.
        idx_sorted = np.argsort(x_sample)
        x_sorted = x_sample[idx_sorted]
        w_sort = weights[idx_sorted]
        x_cdf = np.cumsum(w_sort)
        index_l = next(i for i, x in enumerate(x_cdf) if x > alpha/2)
        index_u = next(i for i, x in enumerate(x_cdf) if x > 1-alpha/2)
        est_bands = np.array([x_sorted[index_l], x_sorted[index_u]])

        return est_mean, est_bands


In [4]:
# Define the state space model.
# Process equation x[k] = sys(k, x[k-1], u[k]):
nx = 4  # number of states
nu = 3  # size of the vector of process noise

# Define the standard deviation of the process noise.
sigma_u = 1*np.array([1e-2, 1e-3, 1e-2])

# Define the degradation model.
def degradation_path(x, t):
    return x[2] / (1 + np.exp(-1*(x[0]+x[1]*t)))

# Process model.
def sys(tk, xkm1, uk):
    xk = np.zeros(nx)
    xk[0] = xkm1[0] + uk[0]
    xk[1] = xkm1[1] + uk[1]
    xk[2] = xkm1[2] + uk[2]
    xk[3] = degradation_path(np.array([xk[0], xk[1], xk[2]]), tk)
    return xk

# Generate system noise.
def gen_sys_noise(Ns=1):
    if Ns == 1:
        sample = np.random.normal(0, sigma_u)
    else:
        sample = np.zeros((nu, Ns))
        for i in range(nu):
            sample[i, :] = np.random.normal(0, sigma_u[i], size=Ns)
    return sample

# PDF of process noise and noise generator function
def p_sys_noise(u):
    return norm.pdf(u, 0, sigma_u)

# Define observation equation.
ny = 1  # number of observations
nv = 1  # size of the vector of observation noise
sigma_v = 5e-2

# Observation equation y[k] = obs(k, x[k], v[k]);
def obs(xk, vk):
    return xk[3] + vk

# PDF of observation noise and noise generator function
def p_obs_noise(v):
    return norm.pdf(v, 0, sigma_v)

# Generate observation noise.
def gen_obs_noise():
    return np.random.normal(0, sigma_v)

In [5]:
def gen_x0(Ns=1, t_0=0):
    x0 = np.zeros((nx, Ns))
    x0[0, :] = np.random.uniform(-6, -4, size=Ns)
    x0[1, :] = np.random.uniform(10/60, 12/60, size=Ns)
    x0[2, :] = np.random.uniform(.9, 1, size=Ns)
    x0[3, :] = degradation_path(np.array([x0[0], x0[1], x0[2]]), t_0)
    return x0

# Observation likelihood.
def p_yk_given_xk(yk, xk):
    return p_obs_noise(yk - obs(xk, 0))

In [6]:
# Filtering items based on the number of rows
unique_item_ids = all_test_data['item_id'].unique()
items_to_estimate = []

for item_id in unique_item_ids:
    item_data = all_test_data[all_test_data['item_id'] == item_id]
    if len(item_data) > 5:
        items_to_estimate.append(item_id)


In [7]:
# Constants and threshold setup
threshold = 0.85
max_RUL = 60  # Assuming you want to limit the RUL predictions to a maximum value
working_after_6_months = {}

for item_id in items_to_estimate:
    item_data = all_test_data[all_test_data['item_id'] == item_id]
    t = item_data['time (months)'].to_numpy()
    y = item_data['crack length (arbitary unit)'].to_numpy().reshape(1, -1)
    T = len(t)  # Number of time steps
    t_0 = t[0] if T > 0 else 0  # Use the first time step or 0 if there are no steps

    # This adjustment is correct if t_0 is fixed and known at the time of pf_class initialization
    pf = pf_class(
        Ns=1000, 
        t=t, 
        nx=nx, 
        gen_x0=lambda Ns, t_0=t[0]: gen_x0(Ns, t_0),  # Adjusted lambda to include t_0 with a default value
        sys=sys, 
        obs=obs,
        p_yk_given_xk=p_yk_given_xk, 
        gen_sys_noise=gen_sys_noise,
        degradation_path=degradation_path
    )

    # Initialize particle filter and perform state estimation
    for k in range(1, T):
        pf.k = k
        pf.state_estimation(y[0, k])
    
    # Predict RUL
    rul_mean, _, _, _, _, _ = pf.rul_prediction(threshold, [T-1], t, max_RUL=max_RUL)
    
    # Check if the item is expected to work beyond 6 months
    is_working_after_6_months = rul_mean > 6
    working_after_6_months[item_id] = is_working_after_6_months


  0%|          | 0/1 [00:00<?, ?it/s]/tmp/ipykernel_72385/2808879871.py:234: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  rul[j, i] = math.floor(ttf_run) + 1 - t[idx_pred_i]
/home/user/anaconda3/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
100%|██████████| 1/1 [00:00<00:00,  3.81it/s]


In [8]:
working_after_6_months

{0: array([ True]),
 1: array([ True]),
 2: array([ True]),
 5: array([ True]),
 6: array([ True]),
 9: array([False]),
 10: array([ True]),
 11: array([ True]),
 13: array([ True]),
 14: array([ True]),
 15: array([ True]),
 17: array([ True]),
 18: array([ True]),
 20: array([ True]),
 21: array([ True]),
 23: array([ True]),
 24: array([ True]),
 25: array([ True]),
 26: array([ True]),
 28: array([ True]),
 29: array([ True]),
 30: array([ True]),
 32: array([ True]),
 33: array([ True]),
 36: array([ True]),
 37: array([ True]),
 38: array([ True]),
 39: array([ True]),
 40: array([ True]),
 42: array([ True]),
 44: array([ True]),
 46: array([ True]),
 47: array([ True]),
 49: array([ True])}